In [ ]:
## generator Sentimenti
import os
from collections import defaultdict
from pandas import read_table
import numpy as np
import math
import datetime

from ekonlpy.sentiment import MPCK

mpck = MPCK()


class NaiveBayesClassifier:
    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = [] # 리스트 자료형 

    def load_corpusData(self, path):
        corpusData = read_table(path, sep=',', header=None, names=None, encoding='utf-8')
        corpusData = np.array(corpusData)

        return corpusData

    def count_words(self, training_set):
        counts = defaultdict(lambda: [0, 0])

        minutes_list = os.listdir('/content/text/') # 폴더 경로 수정

        file_list = minutes_list # 추가 크롤리이 된다면. 

        for dataDate, label in training_set:
            dataDate_without_dot = dataDate.replace('.', '')

            for file in file_list:

                if len(file) <20 : # 해당 pdf 파일 제외한 파일들 입력넘어가기 위한 용도
                  continue
                
                # 날짜처리
                str_datetime = file[24:][:-6].replace('.','-').replace('(','').replace(')','')
                
                if str_datetime[-1]=='-' :
                  str_datetime = str_datetime[:-1]

                format = '%Y-%m-%d' # 2022-1-1 로 string 저장 처리를 위한 함수 
                
                date_time_str = datetime.datetime.strptime(str_datetime, format)
                temp_str=date_time_str.isoformat()[:10]
                
                if dataDate == temp_str: # 해당 날짜처리후 날짜가 같은 것만 학습 ! 변동일 
                  
                  #latin_1
                  #.decode('utf-8')
                  corpus = open('text/'+file, 'r', encoding='utf-8').read()
                 
                  print("process minutes file name : ", file)

                  tokens = mpck.tokenize(corpus)
                  ngrams = mpck.ngramize(tokens)

                  for ngram in ngrams + tokens:
                      counts[ngram][0 if label == 1 else 1] += 1
            
        print(" 단어 갯수세기 부분 완료입니다. counts",counts)
        return counts

    def word_probabilities(self, counts, total_class0, total_class1, k): #[단우][0=상승,1=상승외] = 갯수 반환, 상승, 상승외 , 0.5 
        return [(w, (class0 + k) / (total_class0 + 2 * k), (class1 + k) / (total_class1 + 2 * k))
                for w, (class0, class1) in counts.items()]

[(w, (class0 + k) / (total_class0 + 2 * k), (class1 + k) / (total_class1 + 2 * k)) for w, (class0, class1) in counts.items()] # w = 단어 [0 갯수, 1 갯수]

# 리스트 반환 : 단어 , 단어의 pos [ (단어의 상승 등장 수 + 0.5 ) / (total_class0 + 2 * 0.5 ) ] 


    def train(self, trainfile_path):
        training_set = self.load_corpusData(trainfile_path)
#        print(training_set)
        positive = len([1 for _, label in training_set if label == 1]) #라벨이 1인것 즉 상승의 수 
        negative = len(training_set) - positive # 상승외의 수
        #print( poisi)
        word_counts = self.count_words(training_set)# [단우][0=상승,1=상승외] = 갯수 반환

        self.word_probs = self.word_probabilities(word_counts, positive, negative, self.k)

        # 기존 인코딩 utf-8 -> 수정인코딩 cp949 
        for noOfWord in range(len(self.word_probs)):
            if self.word_probs[noOfWord][1] / self.word_probs[noOfWord][2] > 1:
                with open('/content/positive.csv', 'a', encoding='cp949') as f:
                    f.write(self.word_probs[noOfWord][0] + '\n')
            else:
                with open('/content/negative.csv', 'a', encoding='cp949') as f:
                    f.write(self.word_probs[noOfWord][0] + '\n')


def str_to_date(file) :
  file[24:][:-7].replace('(','').replace(')','').replace('.','-')

model = NaiveBayesClassifier() # 클래스 호줄

model.train(trainfile_path='labeledCallRate.csv') #데이터 라벨용 ! 
